<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from tensorflow import keras

# ResNet34

# The stem component

In [20]:
def stem(inputs):
  '''
  Build the stem component of ResNet34.
  Perform the initial coarse-level feature extraction.

  inputs: the input vector
  '''
  # a conv layer that outputs 64 feature maps
  x = keras.layers.Conv2D(filters=64,
                          kernel_size=7,
                          strides=2,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal",
                          )(inputs)

  # a max pooling layer that pools the 64 feature maps
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

  # following line would have the same effect as the previous max pooling layer
  #x = keras.layers.MaxPooling2D(pool_size=2)(x)


  return x

# The learner component

## Residual block

A block that consists of two convolutional layers with the same number of filters for feature extraction. The input to the residual block will be added to the output of the last conv layer via an identity link.

In [21]:
def residual_block(x, filters):
  '''
  A residual block with an identity link and
  two convolutional layers. All convolutional
  layers have the same number of filters.

  x       :  input into to the residual block
  filters :  nr. of filters of each conv layer
  '''

  # identity link, also referred to as "shortcut"
  shortcut = x

  # apply non-strided 3x3 convolutions for feature extraction
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal"
                          )(x)

  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=1,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal"
                          )(x)

  # add the output feature maps to the input
  # note: padding="same" preserved the size of the width & height
  #       so that adding input and output feature maps can work
  x = keras.layers.Add()([shortcut,x])

  # return the addition of the input & output feature maps
  return x


## Conv block

A block that also consists of two convolutional layers. It will be placed after the last residual block of a residual block. It is responsible for doubling the number of output feature maps for the subsequent residual group.

In [22]:
def conv_block(x, filters):
  '''
  A conv block with two strided convolutional layers that extract
  features, doubles the number of feature maps for the
  next residual group and reduces the size of the feature
  maps.

  x       : input to the conv block
  filters : number of filters
  '''
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=2 ,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal")(x)
  x = keras.layers.Conv2D(filters=filters,
                          kernel_size=3,
                          strides=2,
                          padding="same",
                          activation="relu",
                          kernel_initializer="he_normal")(x)

  # return the output feature maps
  return x

## Residual group

Now, we can create the residual group. The learner component of ResNet34 has 4 residual groups.

In [23]:
def residual_group(x, filters, blocks, conv=True):
  '''
  Create a residual group.

  x       : input to the residual group
  filters : number of filters for the conv layers within the residual blocks
  blocks  : number of residual blocks in the residual group
  conv    : flag that decides whether we need a conv block or not
            (note: all residual groups have one, except the last one.)
  '''

  # create the residual blocks of the residual group
  for _ in range(blocks):
    x = residual_block(x, filters)


  # create the conv block to double the number of feature maps for the
  # upcoming residual group and also reduce the size of the feature maps
  # via a stride of 2
  # (Note: Doubling of the number of feature maps has to be done since each residual
  # group doubles the number of filters in their residual blocks. If we don't
  # do this then the addition of feature maps at the end of a residual block
  # wouldn't work.
  if conv:
    x = conv_block(x, filters * 2)


  # return the output feature maps
  return x

Finally, we can put all pieces together to build the learner component of ResNet34:

In [24]:
def learner(x):
  '''
  Build the learner component of ResNet34.
  Perform the detailed feature extraction and representational learning
  from the extracted coarse features.

  x: input to the learner
  '''

  # create the 1st residual group with 3 residual blocks.
  # Each residual block consists of 2 convolutional layers with 64 filters.
  x = residual_group(x, 64, 3)

  # create the 1st residual group with 3 residual blocks.
  # Each residual block consists of 2 convolutional layers with 128 filters.
  x = residual_group(x, 128, 3)

  # create the 1st residual group with 5 residual blocks.
  # Each residual block consists of 2 convolutional layers with 256 filters.
  x = residual_group(x, 256, 5)

  # create the 1st residual group with 2 residual blocks.
  # Each residual block consists of 2 convolutional layers with 512 filters.
  x = residual_group(x, 512, 2, False)

  # return the final output feature maps (latent space)
  return x

# The task component

In [25]:
def task(x, classes):
  '''
  Build the task component that perform classification.

  x       : input to the task component
  classes : number of classes
  '''

  # global average pooling to reduce and flatten the
  # feature maps into a 1D vector
  x = keras.layers.GlobalAveragePooling2D()(x)

  # softmax layer
  outputs = keras.layers.Dense(units=classes,
                               activation="softmax",
                               kernel_initializer="he_normal")(x)


  # return the probability of each class
  return outputs

In [26]:
inputs = keras.Input(shape=(224,224, 3))

In [27]:
# perform the coarse-level feature extraction
coarse_features = stem(inputs)

# perform the detailed feature extraction & representational learning from the coarse features
detailed_features = learner(coarse_features)

# the task component performing classification
outputs = task(detailed_features, 1000)


model = keras.Model(inputs=inputs, outputs=outputs, name="resnet_34")
model.summary()

Model: "resnet_34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_34 (Conv2D)             (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_34[0][0]']              
                                                                                          